# Sparse PCA Simulation

Let $$S_{p\times p}=\alpha[1\ \cdots\ 1\ 0\ \cdots\ 0]^T[1\ \cdots\ 1\ 0\ \cdots\ 0] + \beta I_p$$ and $$(Z_{n\times p})_{i, j}\sim\mathcal{N}(0, 1).$$
We define $$X_{n,p}\equiv Z_{n,p}S_{p,p}.$$

In [ ]:
import numpy as np
import torch
import random
from matplotlib import pyplot as plt

In [ ]:
p = 50
n = 30
alpha = 0.8
beta = 0.2
sparsity = 0.3
sparse_vector = np.empty((p, 1))
for i in range(p):
    if i < p * sparsity:
        sparse_vector[i][0] = 1
    else:
        sparse_vector[i][0] = 0
S = alpha * sparse_vector @ sparse_vector.T + beta * np.identity(p)

Z = np.empty((n, p))
for i in range(n):
    for j in range(p):
        Z[i][j] = np.random.normal(0, 1)
        
X = Z @ S

sparse_vector = torch.from_numpy(sparse_vector)
S, Z, X = torch.from_numpy(S), torch.from_numpy(Z), torch.from_numpy(X)

In [ ]:
S.shape, Z.shape, X.shape

## Method 1
We want to find $(u, v, w)$ such that 
#### $$(u, v, w)=\underset{(u, v, w)}{\text{argmin}}\ \lVert X-u(v\odot w)^T\rVert_F^2$$
where for any matrix $A$, 
#### $$\lVert A\rVert_F^2 = \sum_{i,j}A_{i,j}^2.$$

In [ ]:
u = torch.randn(n, 1, requires_grad=True)
v = torch.randn(p, 1, requires_grad=True)
w = torch.randn(p, 1, requires_grad=True)

In [ ]:
def objective_function_1(u, v, w):
    A = X - u @ (v * w).T
    return torch.sum(A * A)

In [ ]:
epochs = 10000
step_size = 1e-4
sine_errors = []
l1 = []
iterations = [*range(epochs + 1)]
recovered = v.clone().detach() * w.clone().detach()
cosine_error = torch.sum(recovered * sparse_vector) / (torch.norm(recovered) * torch.norm(sparse_vector))
sine_error = (1 - cosine_error ** 2) ** 0.5
sine_errors.append(sine_error)
l1.append(float(torch.linalg.norm(recovered, ord=1)))
for i in range(epochs):
    objective = objective_function_1(u, v, w)
    if i % 100 == 0:
        print('Epoch ' + str(i) + ": loss = " + str(float(objective)))
    objective.backward()
    with torch.no_grad():
        u -= u.grad * step_size
        v -= v.grad * step_size
        w -= w.grad * step_size
        u.grad.zero_()
        v.grad.zero_()
        w.grad.zero_()
    recovered = v.clone().detach() * w.clone().detach()
    cosine_error = torch.sum(recovered * sparse_vector) / (torch.norm(recovered) * torch.norm(sparse_vector))
    sine_error = (1 - cosine_error ** 2) ** 0.5
    sine_errors.append(sine_error)
    l1.append(float(torch.linalg.norm(recovered, ord=1)))
    v.grad.zero_()
    w.grad.zero_()
print('Epoch ' + str(epochs) + ": loss = " + str(float(objective_function_1(u, v, w))))

In [ ]:
print(v * w)

In [ ]:
plt.plot(iterations, sine_errors, label="sine_error")
plt.plot(iterations, l1, label="l1 norm")
plt.show()

In [ ]:
plt.plot(iterations, sine_errors)
plt.show()

## Method 2
We want to find $(u, v, w)$ such that 
#### $$(u, v, w)=\underset{(u, v, w)}{\text{argmin}}\ \lVert X-Xu(v\odot w)^T\rVert_F^2.$$

In [ ]:
u = torch.randn(p, 1, requires_grad=True)
v = torch.randn(p, 1, requires_grad=True)
w = torch.randn(p, 1, requires_grad=True)

In [ ]:
def objective_function_2(u, v, w):
    A = X - (X.double() @ u.double()).double() @ (v * w).T.double()
    return torch.sum(A * A)

In [ ]:
epochs = 100000
step_size = 5e-7
sine_errors = []
l1 = []
iterations = [*range(epochs + 1)]
recovered = v.clone().detach() * w.clone().detach()
cosine_error = torch.sum(recovered * sparse_vector) / (torch.norm(recovered) * torch.norm(sparse_vector))
sine_error = (1 - cosine_error ** 2) ** 0.5
sine_errors.append(sine_error)
l1.append(float(torch.linalg.norm(recovered, ord=1)))
for i in range(epochs):
    objective = objective_function_2(u, v, w)
    if i % 1000 == 0:
        print('Epoch ' + str(i) + ": loss = " + str(float(objective)))
    objective.backward()
    with torch.no_grad():
        u -= u.grad * step_size
        v -= v.grad * step_size
        w -= w.grad * step_size
        u.grad.zero_()
        v.grad.zero_()
        w.grad.zero_()
    recovered = v.clone().detach() * w.clone().detach()
    cosine_error = torch.sum(recovered * sparse_vector) / (torch.norm(recovered) * torch.norm(sparse_vector))
    sine_error = (1 - cosine_error ** 2) ** 0.5
    sine_errors.append(sine_error)
    l1.append(float(torch.linalg.norm(recovered, ord=1)))
print('Epoch ' + str(epochs) + ": loss = " + str(float(objective_function_2(u, v, w))))

In [ ]:
print(v * w)

In [ ]:
plt.plot(iterations, sine_errors, label="sine_errors")
plt.plot(iterations, l1, label="l1")
plt.show()

In [ ]:
plt.plot(iterations, sine_errors)
plt.show()

## Method 3
We want to find $(u, v, w)$ such that 
#### $$(u, v, w)=\underset{(u, v, w)}{\text{argmin}}\ \lVert X-u(v\odot v - w\odot w)^T\rVert_F^2.$$

In [ ]:
u = torch.randn(n, 1, requires_grad=True)
v = torch.randn(p, 1, requires_grad=True)
w = torch.randn(p, 1, requires_grad=True)

In [ ]:
def objective_function_3(u, v, w):
    A = X - u @ (v * v - w * w).T
    return torch.sum(A * A)

In [ ]:
epochs = 10000
step_size = 1e-4
sine_errors = []
l1 = []
iterations = [*range(epochs + 1)]
recovered = v.clone().detach() * v.clone().detach() - w.clone().detach() * w.clone().detach()
cosine_error = torch.sum(recovered * sparse_vector) / (torch.norm(recovered) * torch.norm(sparse_vector))
sine_error = (1 - cosine_error ** 2) ** 0.5
sine_errors.append(sine_error)
l1.append(float(torch.linalg.norm(recovered, ord=1)))
for i in range(epochs):
    objective = objective_function_3(u, v, w)
    if i % 100 == 0:
        print('Epoch ' + str(i) + ": loss = " + str(float(objective)))
    objective.backward()
    with torch.no_grad():
        u -= u.grad * step_size
        v -= v.grad * step_size
        w -= w.grad * step_size
        u.grad.zero_()
        v.grad.zero_()
        w.grad.zero_()
    recovered = v.clone().detach() * v.clone().detach() - w.clone().detach() * w.clone().detach()
    cosine_error = torch.sum(recovered * sparse_vector) / (torch.norm(recovered) * torch.norm(sparse_vector))
    sine_error = (1 - cosine_error ** 2) ** 0.5
    sine_errors.append(sine_error)
    l1.append(float(torch.linalg.norm(recovered, ord=1)))
print('Epoch ' + str(epochs) + ": loss = " + str(float(objective_function_3(u, v, w))))

In [ ]:
print(v * v - w * w)

In [ ]:
plt.plot(iterations, sine_errors, label="sine_errors")
plt.plot(iterations, l1, label="l1")
plt.show()

In [ ]:
plt.plot(iterations, sine_errors)

## Method 4
We want to find $(u, v, w)$ such that 
#### $$(u, v, w)=\underset{(u, v, w)}{\text{argmin}}\ \lVert X-Xu(v\odot v - w\odot w)^T\rVert_F^2.$$

In [ ]:
u = torch.randn(p, 1, requires_grad=True)
v = torch.randn(p, 1, requires_grad=True)
w = torch.randn(p, 1, requires_grad=True)

In [ ]:
def objective_function_4(u, v, w):
    A = X - X.double() @ u.double() @ (v * v - w * w).T.double()
    return torch.sum(A * A)

In [ ]:
epochs = 100000
step_size = 5e-7
sine_errors = []
l1 = []
iterations = [*range(epochs + 1)]
recovered = v.clone().detach() * v.clone().detach() - w.clone().detach() * w.clone().detach()
cosine_error = torch.sum(recovered * sparse_vector) / (torch.norm(recovered) * torch.norm(sparse_vector))
sine_error = (1 - cosine_error ** 2) ** 0.5
sine_errors.append(sine_error)
l1.append(float(torch.linalg.norm(recovered, ord=1)))
for i in range(epochs):
    objective = objective_function_4(u, v, w)
    if i % 1000 == 0:
        print('Epoch ' + str(i) + ": loss = " + str(float(objective)))
    objective.backward()
    with torch.no_grad():
        u -= u.grad * step_size
        v -= v.grad * step_size
        w -= w.grad * step_size
        u.grad.zero_()
        v.grad.zero_()
        w.grad.zero_()
    recovered = v.clone().detach() * v.clone().detach() - w.clone().detach() * w.clone().detach()
    cosine_error = torch.sum(recovered * sparse_vector) / (torch.norm(recovered) * torch.norm(sparse_vector))
    sine_error = (1 - cosine_error ** 2) ** 0.5
    sine_errors.append(sine_error)
    l1.append(float(torch.linalg.norm(recovered, ord=1)))
print('Epoch ' + str(epochs) + ": loss = " + str(float(objective_function_4(u, v, w))))

In [ ]:
print(v * v - w * w)

In [ ]:
plt.plot(iterations, sine_errors, label="sine_errors")
plt.plot(iterations, l1, label="l1")
plt.show()

In [ ]:
plt.plot(iterations, sine_errors)
plt.show()

### Hadamard Parametrization
We want to maximize 
#### $$(u\odot v)^TS(u\odot v)$$
where
#### $$S=\frac{1}{n}X^TX$$
under the constraint
#### $$\lVert u\odot v\rVert = 1$$

In [ ]:
u = torch.randn(p, 1)
v = torch.randn(p, 1)
norm = torch.norm(u * v)
u = u / (norm ** 0.5)
v = v / (norm ** 0.5)

In [ ]:
epochs = 200000
step_size = 1e-5
sine_errors = []
l1 = []
iterations = [*range(epochs + 1)]
recovered = u.clone().detach() * v.clone().detach()
cosine_error = torch.sum(recovered * sparse_vector) / (torch.norm(recovered) * torch.norm(sparse_vector))
sine_error = (1 - cosine_error ** 2) ** 0.5
sine_errors.append(sine_error)
l1.append(float(torch.linalg.norm(recovered, ord=1)))
for i in range(epochs):
    if i % 1000 == 0:
        objective = ((u * v).T.double() @ S.double() @ (u * v).double())[0][0]
        print('Epoch ' + str(i) + ": objective = " + str(float(objective)))
    du = v.double() * ((S + S.T).double() @ (u * v).double())
    dv = u.double() * ((S + S.T).double() @ (u * v).double())
    u += du * step_size
    v += dv * step_size
    norm = torch.norm(u * v)
    u = u / (norm ** 0.5)
    v = v / (norm ** 0.5)
    recovered = u.clone().detach() * v.clone().detach()
    cosine_error = torch.sum(recovered * sparse_vector) / (torch.norm(recovered) * torch.norm(sparse_vector))
    sine_error = (1 - cosine_error ** 2) ** 0.5
    sine_errors.append(sine_error)
    l1.append(float(torch.linalg.norm(recovered, ord=1)))
objective = ((u * v).T.double() @ S.double() @ (u * v).double())[0][0]
print('Epoch ' + str(epochs) + ": objective = " + str(float(objective)))

In [ ]:
print(u * v)

In [ ]:
plt.plot(iterations, sine_errors, label="sine_errors")
plt.plot(iterations, l1, label="l1")
plt.show()

In [ ]:
plt.plot(iterations, sine_errors)
plt.show()